apt install postgresql

![Challenges of Postgres](./images/ChallengesOfPostgres.png)

Store list of largest cities: https://en.wikipedia.org/wiki/List_of_largest_cities  
Focus only on 4 columns: City, Country, Population, Area

![DatabaseDesignProcess](./images/DatabaseDesignProcess.png)

$ sudo -u postgres psql

Inside Postgres(#):  
CREATE DATABASE db;  
CREATE USER debayan WITH PASSWORD '12345';  
exit

$ psql -h localhost -p 5432 -U debayan db 

DataTypes in Postgres - https://www.postgresql.org/docs/current/datatype.html

CREATE TABLE cities(  
    name VARCHAR(50),  
    country VARCHAR(50),  
    population INTEGER,  
    area INTEGER  
);  

DB: \l  
Connect to a DB: \c db  
Tables: \d  
SELECT * FROM cities;  

### INSERT INTO TABLE

INSERT INTO cities(name, country, population, area)  
VALUES ('Tokyo', 'Japan', 38505000, 8223);  
/* name, population, country, area - can be changed posution and we need to put the data accordingly in VALUES*/

### Insert multiple rows in one go:  
INSERT INTO cities(name, country, population, area)  
VALUES 
    ('Delhi', 'India', 28125000, 2240),  
    ('Shanghai', 'China', 22125000, 4015),  
    ('Sao Paulo', 'Brazil', 20935000, 3043);   



### SELECT

SELECT * FROM cities;  
SELECT name, country FROM cities;

SQL is not just about pulling raw data out of a table. We can write SQL to transform or process data we receive it

Population Density = Population / Area  

SELECT name, population/area AS density  
FROM cities; 

/* Other Math Operators: +, -, *, /, ^(exponent), |/(Square Root), @(Absolute Value), %(Remainder) */

String Operators and Functions:  
Join 2 strings: ||, CONCAT()  
Gives a lower case string: LOWER()  
Gives a upper case string: UPPER()  
Gives number of characters in a string: LENGTH()    

SELECT name || ', ' || country AS location  FROM cities;

### Filtering

SELECT name, area FROM cities WHERE area > 4000;  
**Comparison Math Operators**: =(unlike == in most programming languages), >, <, >=, <=, IN, NOT IN, BETWEEN, !=, <>(Not Equal)

BETWEEN 2000 and 4000

WHERE name IN('Delhi', 'Shanghai')

Compound Check:  
area NOT IN(3043, 8223) AND name = 'Delhi';

WHERE manufacturer IN('Apple', 'Samsung')  
WHERE manufacturer = 'Apple' or manufacturer = 'Samsung'  

### WHERE Clause for Calculations

SELECT name, population/area AS population_density  
FROM cities  
WHERE population / area > 6000;

SELECT name, price*units_sold AS total_revenue FROM phones WHERE price*units_sold > 1000000;

/* We cannot refer total_revenue in the WHERE clause we need to recalculate price*units_sold */

### UPDATING AND DELETING RECORDS

UPDATE cities SET population = 39505000 WHERE name = 'Tokyo';

DELETE FROM cities WHERE name = 'Tokyo';

![PhotoSharingApp](./images/PhotoSharingApp.png)

4 Different kinds of relationships:  
1. One to Many Relationship: A user has many photos  
2. many to One: A photo has one user  
3. Many to Many  
4. One to One  

Primary Key, Foreign Key  
![Primary_Foreign_Key](./images/PrimaryKey_ForeignKey.png)

### The "many" side of the relationship gets the foreign key column

foreign key notation - example: photo_id

CREATE TABLE users(  
    id SERIAL PRIMARY KEY,  
    username VARCHAR(50)  
);  

CREATE TABLE photos(  
    id SERIAL PRIMARY KEY,  
    url VARCHAR(200),  
    user_id INTEGER REFERENCES users(id)
)  

INSERT INTO photos(url, user_id)

SELECT * FROM photos WHERE user_id = 4;

![OnDelete](./images/OnDelete.png)

### ON DELETE CASCADE

CREATE TABLE photos(  
    id SERIAL PRIMARY KEY,  
    url VARCHAR(200),  
    user_id INTEGER REFERENCES users(id) ON DELETE CASCADE
)  
whenever we delete a user from users table all the photos associated with the user id will be deleted automatically

### Joins and Aggregation

![Join_Aggregation](./images/JoinAggregation.png)

comments: id, contents, user_id, photo_id  
users: id, username  

SELECT contents, username  
FROM comments   
JOIN users ON users.id = comments.user_id;


We can flip comments and users  

Join X and Y - FROM [X] JOIN [Y] - create new row and match the two rows of two tables by the statement ON users.id = comments.user_id;  

![Join](./images/Join.png)

Table order between 'FROM' and 'JOIN' frequently makes a difference  
We must give context if column names collide  
Tables can be renamed using the 'AS' keyword  
There are a few kinds of joins!


SELECT id FROM .. JOIN will not work if the resulting table contains two columns having the same name "id"

RENAME if needed if two columns have the same name in the output - Eg, SELECT users.id, comments.id  

SELECT comments.id AS comments_id

SELECT comments.id AS comment_id, p.id  
FROM photos AS p  
JOIN comments ON p.id = comments.photo_id  

FROM photos - LEFT  
[LEFT/RIGHT/FULL] JOIN comments - RIGHT  

Four Types of Joins:  
1. INNER JOIN: Intersection  
2. LEFT OUTER JOIN: Intersection + All elements in the LEFT - FROM photos, Missing entries 'NULL'  
3. RIGHT OUTER JOIN: Intersection + All elements in the RIGHT - FROM comments, Missing entries 'NULL'  
4. Full Join: Take everything from LEFT(photos) + Intersection + Everything from RIGHT(comments), Missing entries 'NULL'

### WHERE with JOIN

FROM -> JOIN -> WHERE  

SELECT url, contents  
FROM comments  
JOIN photos ON photos.id = comments.photo_id  
WHERE comments.user_id = photos.user_id;  

We can add multiple JOINs:

SELECT url, contents, username  
FROM comments  
JOIN photos ON photos.id = comments.photo_id  
JOIN users ON users.id = comments.users_id AND users.id = photos.user_id  

### Aggregating and Grouping 

![Aggregating_Grouping](./images/%20AggregatingGrouping.png)

### GROUP BY

Find unique values - group by - will create a IMAGINARY column - "GROUPED BY user_id"  
Multiple block of rows - grouped by user_id  
WE CANNOT select OTHER COLUMNS here - We can only select the GROUP BY column or by using aggregate function


![GROUP_BY](./images/GROUP_BY.png)

Aggregate Functions:  

COUNT(), SUM(), AVG(), MIN(), MAX()

SELECT MAX(id) FROM comments;  
SELECT MAX(id), id FROM comments; - We cannot use aggregate and column name at the same time without using GROUP BY

![Aggregate_GROUP_BY](./images/Aggregate_GROUP.png)

SELECT user_id, COUNT(id) AS num_comments_created  
FROM comments  
GROUP BY user_id;

SELECT COUNT(user_id) FROM photos; // NULL values will not be counted- use "*" will also count NULL values

FROM > JOIN > WHERE > GROUP BY > HAVING

HAVING: Filter the set of groups, HAVING should always have a GROUP BY

SELECT photo_id, COUNT(*)  
FROM comments  
WHERE photo_id < 3  
GROUP BY photo_id  
HAVING COUNT(*) > 2;

Indexing - chatGPT